# Determine burst sizes from single-cell fluorescent data

Data from the seqFISH (transcriptomic fluorescence assays) paper "High-resolution spatial multi-omics reveals cell-type specific nuclear compartments" by Takei *et al.* (2023).

Preprint: https://www.biorxiv.org/content/10.1101/2023.05.07.539762v1

Raw Data: https://outlook.office.com/mail/safelink.html?url=https://zenodo.org/record/7693825&corid=cddd4553-1e93-1bbb-63f9-c163d915e8dc 

In [8]:
import os
import numpy as np
import pandas as pd
import pickle

In [9]:
# functions 

def get_burst_size_per_cell(count_df):
    ''' Return cell by gene count matrix with burst sizes, which is the average of intensity of intronic spots
        observed per gene per cell. 
    '''
    
    num_genes = len(count_df['geneID'].unique())
    num_cells = len(count_df['unique_id'].unique())
    df_ = count_df.groupby(['unique_id','geneID'])['intensity'].mean().reset_index(name='mean_burst_size')
    
    genes = count_df['geneID'].unique()
    cells = count_df['unique_id'].unique()
    genes_long = list(genes)*len(cells)

    cells_long = []

    for c in cells:
        cells_long+=[c]*len(genes)
    
    all_genes_cells = pd.DataFrame({'geneID': genes_long,'unique_id' : cells_long})
    df_merge = all_genes_cells.merge(df_,left_on=['geneID','unique_id'],right_on=['geneID','unique_id'],
                                how='left').fillna(0)
    
    count_array = np.zeros((num_cells,num_genes))
    
    for i,ID in enumerate(count_df['unique_id'].unique()):
        counts_ = df_merge[df_merge['unique_id']==ID]
        count_array[i,:] = counts_['mean_burst_size']
    
        
    return(count_array)


def get_count_matrix(count_df):
    ''' Given a count data frame with each count, return cell by gene matrix 
    with number of times each gene was observed in each cell (not weighted by intensity).
    '''
    
    num_genes = len(count_df['geneID'].unique())
    num_cells = len(count_df['unique_id'].unique())
    df_ = count_df.groupby(['unique_id','geneID']).size().reset_index(name='counts')
    
    genes = count_df['geneID'].unique()
    cells = count_df['unique_id'].unique()
    genes_long = list(genes)*len(cells)

    cells_long = []

    for c in cells:
        cells_long+=[c]*len(genes)
    
    all_genes_cells = pd.DataFrame({'geneID': genes_long,'unique_id' : cells_long})
    df_merge = all_genes_cells.merge(df_,left_on=['geneID','unique_id'],right_on=['geneID','unique_id'],
                                how='left').fillna(0)
    
    count_array = np.zeros((num_cells,num_genes))
    
    for i,ID in enumerate(count_df['unique_id'].unique()):
        counts_ = df_merge[df_merge['unique_id']==ID]
        count_array[i,:] = counts_['counts']
    
        
    return(count_array)


def get_adjusted_count_matrix(count_df):
    ''' Return cell by gene count matrix with intronic counts adjusted by intensity value.
    '''
    
    count_df['adjusted_intron_counts'] = np.ceil(count_df['intensity'].values)

    num_genes = len(count_df['geneID'].unique())
    num_cells = len(count_df['unique_id'].unique())
    df_ = count_df.groupby(['unique_id','geneID'])['adjusted_intron_counts'].sum().reset_index(name='adjusted_intron_counts')
    
    genes = count_df['geneID'].unique()
    cells = count_df['unique_id'].unique()
    genes_long = list(genes)*len(cells)

    cells_long = []

    for c in cells:
        cells_long+=[c]*len(genes)
    
    all_genes_cells = pd.DataFrame({'geneID': genes_long,'unique_id' : cells_long})
    df_merge = all_genes_cells.merge(df_,left_on=['geneID','unique_id'],right_on=['geneID','unique_id'],
                                how='left').fillna(0)
    
    count_array = np.zeros((num_cells,num_genes))
    
    for i,ID in enumerate(count_df['unique_id'].unique()):
        counts_ = df_merge[df_merge['unique_id']==ID]
        count_array[i,:] = counts_['adjusted_intron_counts']
    
        
    return(count_array)

In [11]:
# load in data
count_dir = '/home/tara/maria/scBIVI/review/data/takei'
es_intron_raw = pd.read_csv(os.path.join(count_dir,'E14-replicate1-intron-seqFISH-ch2.csv'))
es_exon_raw = pd.read_csv(os.path.join(count_dir,'E14-replicate1-mRNA-seqFISH-ch1.csv'))

es_intron_raw2 = pd.read_csv(os.path.join(count_dir,'E14-replicate2-intron-seqFISH-ch2.csv'))
es_exon_raw2 = pd.read_csv(os.path.join(count_dir,'E14-replicate2-mRNA-seqFISH-ch1.csv'))


# add unique ids
# each field of fov contains different cells, but cell indexing begins again at each fov
es_intron_raw['unique_id'] = 'cell' + es_intron_raw['cellID'].astype(str)+'_fov'+es_intron_raw['fov'].astype('str')
es_exon_raw['unique_id'] = 'cell' + es_exon_raw ['cellID'].astype(str)+'_fov'+es_exon_raw['fov'].astype('str')
es_intron_raw2['unique_id'] = 'cell' + es_intron_raw2['cellID'].astype(str)+'_fov'+es_intron_raw2['fov'].astype('str')
es_exon_raw2['unique_id'] = 'cell' + es_exon_raw2['cellID'].astype(str)+'_fov'+es_exon_raw2['fov'].astype('str')

In [14]:
# get burst sizes for all introns
es_burst_sizes_all_1 = get_burst_size_per_cell(es_intron_raw)
es_burst_sizes_all_2 = get_burst_size_per_cell(es_intron_raw2)

# get adjusted intronic counts -- adjust the intron spot by its intensity rounded to an integer
es_adjusted_intron_counts1_all = get_adjusted_count_matrix(es_intron_raw)
es_adjusted_intron_counts2_all = get_adjusted_count_matrix(es_intron_raw2)

# get intronic counts -- should only be ONE or TWO
es_intron_counts1_all = get_count_matrix(es_intron_raw)
es_intron_counts2_all = get_count_matrix(es_intron_raw2)

# get exonic counts
es_exon_counts1_all = get_count_matrix(es_exon_raw)
es_exon_counts2_all = get_count_matrix(es_exon_raw2)

In [15]:
# save
es_bs_dict_1 = {'burst_size' : es_burst_sizes_all_1,
           'gene_name' : es_intron_raw['geneID'].unique(),
           'cell_id' : es_intron_raw['unique_id'].unique(),
           'adj_intron_counts' : es_adjusted_intron_counts1_all,
           'intron_counts' : es_intron_counts1_all, 
           'exon_counts' : es_exon_counts1_all,
           'cell_id_exon' : es_exon_raw['unique_id'].unique(),
           'gene_name_exon': es_exon_raw['geneID'].unique() }

es_bs_dict_2 = {'burst_size' : es_burst_sizes_all_2,
           'gene_name' : es_intron_raw2['geneID'].unique(),
           'adj_intron_counts' : es_adjusted_intron_counts2_all,
           'intron_counts' : es_intron_counts2_all, 
           'exon_counts' : es_exon_counts2_all, 
           'cell_id_exon' : es_exon_raw2['unique_id'].unique(),
           'gene_name_exon': es_exon_raw2['geneID'].unique() }

# save! 
with open('./data/takei/E14_rep1_seqFISH_burst_sizes_all','wb') as file:
    pickle.dump(es_bs_dict_1,file,protocol=pickle.HIGHEST_PROTOCOL)
with open('./data/takei/E14_rep2_seqFISH_burst_sizes_all','wb') as file:
    pickle.dump(es_bs_dict_2,file,protocol=pickle.HIGHEST_PROTOCOL)

SyntaxError: invalid syntax (<ipython-input-16-4ebbbfa2ec9d>, line 1)